导入包

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding,Dense,Dropout,Bidirectional,LSTM,GRU,Conv1D,Layer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc

读取数据

In [2]:
with  open("data\ST-train.fa") as f:
    ST_train = f.readlines()
    ST_train = [s.strip() for s in ST_train]
with  open("data\ST-test.fa") as f:
    ST_test = f.readlines()
    ST_test = [s.strip() for s in ST_test]
with  open("data\Y-train.fa") as f:
    Y_train = f.readlines()
    Y_train = [s.strip() for s in Y_train]
with  open("data\Y-test.fa") as f:
    Y_test = f.readlines()
    Y_test = [s.strip() for s in Y_test]
print(len(ST_train),len(ST_test),len(Y_train),len(Y_test))

17232 4316 324 84


提取序列

In [3]:
def remove_name(data):
    data_new = []
    for i in range(1,len(data),2):
        data_new.append(data[i])
    return data_new

In [4]:
ST_train_x = remove_name(ST_train)
ST_test_x = remove_name(ST_test)
Y_train_x = remove_name(Y_train)
Y_test_x = remove_name(Y_test)
print(len(ST_train_x),len(ST_train_x[0]))
print(len(ST_test_x),len(ST_test_x[0]))
print(len(Y_train_x),len(Y_train_x[0]))
print(len(Y_test_x),len(Y_test_x[0]))

8616 33
2158 33
162 33
42 33


定义标签

In [5]:
ST_train_y = np.concatenate([np.ones((int(len(ST_train_x)/2),)), np.zeros((int(len(ST_train_x)/2),))], axis=0)  #竖向拼接
ST_test_y = np.concatenate([np.ones((int(len(ST_test_x)/2),)), np.zeros((int(len(ST_test_x)/2),))], axis=0)
print(ST_train_y.shape,ST_test_y.shape)
Y_train_y = np.concatenate([np.ones((int(len(Y_train_x)/2),)), np.zeros((int(len(Y_train_x)/2),))], axis=0)  #竖向拼接
Y_test_y = np.concatenate([np.ones((int(len(Y_test_x)/2),)), np.zeros((int(len(Y_test_x)/2),))], axis=0)
print(Y_train_y.shape,Y_test_y.shape)

(8616,) (2158,)
(162,) (42,)


数字编码

In [6]:
def encode_matrix(seq_matrix):
    """将字符编码为整数
    """
    ind_to_char = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M','N','P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'X']
    char_to_ind = {char: i for i, char in enumerate(ind_to_char)}
    return [[char_to_ind[i] for i in s] for s in seq_matrix]

In [7]:
ST_train_x = encode_matrix(ST_train_x)
ST_test_x = encode_matrix(ST_test_x)
Y_train_x = encode_matrix(Y_train_x)
Y_test_x = encode_matrix(Y_test_x)
ST_train_x = np.array(ST_train_x)
ST_test_x = np.array(ST_test_x)
Y_train_x = np.array(Y_train_x)
Y_test_x = np.array(Y_test_x)
print(ST_train_x.shape,ST_test_x.shape,Y_train_x.shape,Y_test_x.shape)

(8616, 33) (2158, 33) (162, 33) (42, 33)


评价指标

In [8]:
#定义SN、SP、ACC、MCC
def sn_sp_acc_mcc(true_label,predict_label,pos_label=1):
    import math
    pos_num = np.sum(true_label==pos_label)
    print('pos_num=',pos_num)
    neg_num = true_label.shape[0]-pos_num
    print('neg_num=',neg_num)
    tp =np.sum((true_label==pos_label) & (predict_label==pos_label))
    print('tp=',tp)
    tn = np.sum(true_label==predict_label)-tp
    print('tn=',tn)
    sn = tp/pos_num
    sp = tn/neg_num
    acc = (tp+tn)/(pos_num+neg_num)
    fn = pos_num - tp
    fp = neg_num - tn
    print('fn=',fn)
    print('fp=',fp)
    
    tp = np.array(tp,dtype=np.float64)
    tn = np.array(tn,dtype=np.float64)
    fp = np.array(fp,dtype=np.float64)
    fn = np.array(fn,dtype=np.float64)
    mcc = (tp*tn-fp*fn)/(np.sqrt((tp+fn)*(tp+fp)*(tn+fp)*(tn+fn)))
    return sn,sp,acc,mcc

注意力机制

In [9]:
class Attention3d(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention3d, self).__init__(**kwargs)
    def get_config(self):
         config = {"W_regularizer":self.W_regularizer,
                   "b_regularizer":self.b_regularizer,"W_constraint":self.W_constraint,"b_constraint":self.b_constraint,
                    "bias":self.bias,"step_dim":self.step_dim,"features_dim":self.features_dim}
         base_config = super(Attention3d, self).get_config()
         return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=initializers.get('glorot_uniform'),
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

构建模型

In [10]:
def define_model():
    maxlen = 33
    max_features = 21
    embedding_dims = 64
    class_num = 1
    last_activation = 'sigmoid'
    input = Input((maxlen,))
    embedding = Embedding(max_features, embedding_dims, input_length=maxlen)(input)

    x = Bidirectional(GRU(64, return_sequences=True))(embedding)
    x = Bidirectional(GRU(32, return_sequences=True))(x)
#     x = Bidirectional(GRU(16, return_sequences=True))(x)
    x = Dropout(0.5)(x)
    x = Attention3d(maxlen)(x)

    t = Dense(16,activation='relu')(x)
    output = Dense(class_num, activation=last_activation)(t)
    model = Model(inputs=input, outputs=output)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
    return model

In [ ]:
# 早停法
checkpoint = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                              min_delta=0,
                                              patience=5,
                                              verbose=1,
                                              mode='auto')

交叉验证

In [ ]:
#交叉验证
def Kfold(data_x,data_y,K):
    kfold = StratifiedKFold(n_splits=K, shuffle=True, random_state=7)
    sn_score = []
    sp_score = []
    acc_score = []
    mcc_score = []
    auc_score = []

    for i,(train, test) in enumerate(kfold.split(data_x, data_y)):
        print('\n\n%d'%i)
        
        model = define_model()
        model.fit(data_x[train],data_y[train],epochs=200,validation_data=(data_x[test],data_y[test]),shuffle=True,callbacks=[checkpoint],verbose=1)
        res = model.predict(data_x[test])
        pred = np.squeeze(res,axis=-1)
        f = pred>0.5
        pred[f]=1
        pred[pred<0.6]=0
        sn_sp_acc_mcc_5fold = sn_sp_acc_mcc(data_y[test],pred,pos_label=1)
        print(sn_sp_acc_mcc_5fold)
        FPR,TPR,threshold = roc_curve(data_y[test],model.predict(data_x[test]),pos_label=1)
        AUC = auc(FPR,TPR)
        print(AUC)
        sn_score.append(round(sn_sp_acc_mcc_5fold[0],4))
        sp_score.append(round(sn_sp_acc_mcc_5fold[1],4))
        acc_score.append(round(sn_sp_acc_mcc_5fold[2],4))
        mcc_score.append(round(sn_sp_acc_mcc_5fold[3],4))
        auc_score.append(round(AUC,4))
        
    return sn_score,sp_score,acc_score,mcc_score,auc_score

In [ ]:
ST_5fold_sn,ST_5fold_sp,ST_5fold_acc,ST_5fold_mcc,ST_5fold_auc = Kfold(ST_train_x,ST_train_y,5)

In [ ]:
ST_5fold_sn_mean = round(np.mean(ST_5fold_sn),4)
ST_5fold_sp_mean = round(np.mean(ST_5fold_sp),4)
ST_5fold_acc_mean = round(np.mean(ST_5fold_acc),4)
ST_5fold_mcc_mean = round(np.mean(ST_5fold_mcc),4)
ST_5fold_auc_mean = round(np.mean(ST_5fold_auc),4)

In [ ]:
with open("result/ST_5fold_result.txt","w") as f:
    f.write("ST_5fold_sn_mean:"+str(ST_5fold_sn_mean)+'\n')
    f.write("ST_5fold_sp_mean:"+str(ST_5fold_sp_mean)+'\n')
    f.write("ST_5fold_acc_mean:"+str(ST_5fold_acc_mean)+'\n')
    f.write("ST_5fold_mcc_mean:"+str(ST_5fold_mcc_mean)+'\n')
    f.write("ST_5fold_auc_mean:"+str(ST_5fold_auc_mean)+'\n')

In [ ]:
Y_5fold_sn,Y_5fold_sp,Y_5fold_acc,Y_5fold_mcc,Y_5fold_auc = Kfold(Y_train_x,Y_train_y,5)

In [ ]:
Y_5fold_sn_mean = round(np.mean(Y_5fold_sn),4)
Y_5fold_sp_mean = round(np.mean(Y_5fold_sp),4)
Y_5fold_acc_mean = round(np.mean(Y_5fold_acc),4)
Y_5fold_mcc_mean = round(np.mean(Y_5fold_mcc),4)
Y_5fold_auc_mean = round(np.mean(Y_5fold_auc),4)

In [ ]:
with open("result/Y_5fold_result.txt","w") as f:
    f.write("Y_5fold_sn_mean:"+str(Y_5fold_sn_mean)+'\n')
    f.write("Y_5fold_sp_mean:"+str(Y_5fold_sp_mean)+'\n')
    f.write("Y_5fold_acc_mean:"+str(Y_5fold_acc_mean)+'\n')
    f.write("Y_5fold_mcc_mean:"+str(Y_5fold_mcc_mean)+'\n')
    f.write("Y_5fold_auc_mean:"+str(Y_5fold_auc_mean)+'\n')

独立测试

In [13]:
ST_model = define_model()

In [14]:
ST_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 33)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 33, 64)            1344      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 33, 128)           49920     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 33, 64)            31104     
_________________________________________________________________
dropout_1 (Dropout)          (None, 33, 64)            0         
_________________________________________________________________
attention3d_1 (Attention3d)  (None, 64)                97        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                1040

In [ ]:
ST_model.fit(ST_train_x,ST_train_y,epochs=200,validation_data=(ST_test_x,ST_test_y),shuffle=True,callbacks=[checkpoint],verbose=1)

In [30]:
ST_model.save_weights('model/ST_model.h5')

In [18]:
Y_model = define_model()

In [19]:
Y_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 33)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 33, 64)            1344      
_________________________________________________________________
bidirectional_4 (Bidirection (None, 33, 128)           49920     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 33, 64)            31104     
_________________________________________________________________
dropout_2 (Dropout)          (None, 33, 64)            0         
_________________________________________________________________
attention3d_2 (Attention3d)  (None, 64)                97        
_________________________________________________________________
dense_4 (Dense)              (None, 16)                1040

In [ ]:
Y_model.fit(Y_train_x,Y_train_y,epochs=200,validation_data=(Y_test_x,Y_test_y),shuffle=True,callbacks=[checkpoint],verbose=1)

In [31]:
Y_model.save_weights('model/Y_model.h5')